# Projet

## Objectifs :
- Mise en place d'un système de recommandation (https://en.wikipedia.org/wiki/Recommender_system) bien commenté en Python
- Rédaction du rapport de projet.

## Projet (partie 1)

L'objectif de ce projet est de recommander des images en fonction des préférences de l'utilisateur. Vous disposez de trois séances pratiques pour construire ce système. Vous devez vous assurer que toutes les tâches liées à l'acquisition, l'annotation, l'analyse et la visualisation des données sont automatisées.

Les principales tâches du projet sont présentées ci-dessous :
1. Collecte de données
2. Étiquetage et annotation
3. Analyses de données
4. Visualisation des données
5. Système de recommandation
6. Tests
7. Rapport

### Collecte de données
Vous devez collecter et télécharger un ensemble d'images. Vous avez les tâches suivantes à programmer, en automatisant le processus autant que possible :
1. Créer un dossier appelé `images`
2. Télécharger les images sous licence ouverte dans le dossier images (minimum 100 images).
3. Enregistrez les métadonnées de chaque image comme la taille de l'image, le format de l'image ( jpeg, .png, etc.), l'orientation de l'image (paysage, portrait, carré, etc.), date de création, modèle d'appareil photo, etc. dans un ou plusieurs fichiers JSON. Vous pouvez utiliser les informations Exif présentes dans les fichiers d'images.

1. Configuration initiale

In [162]:
import sys
import os
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import shutil
from urllib.parse import urlparse, unquote

IMAGES_DIR = 'images'
if not os.path.exists(IMAGES_DIR):
    os.makedirs(IMAGES_DIR)

endpoint_url = "https://query.wikidata.org/sparql"

2. Fonction pour exécuter la requête SPARQL

In [163]:
def get_sparql_results(endpoint_url, query):
    """
    Exécute la requête SPARQL et renvoie les résultats.
    
    :param endpoint_url: URL de l'endpoint SPARQL
    :param query: requête SPARQL
    :return: résultats de la requête
    """
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

3. Script principal pour la collecte des données

In [ ]:
# Définition de la requête SPARQL (LIMIT à modfier selon le besoin)
query = """
SELECT DISTINCT ?grandeville ?grandevilleLabel ?pays ?paysLabel ?image {
    ?grandeville wdt:P31 wd:Q1549591;
    wdt:P17 ?pays;
    wdt:P18 ?image.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
}
LIMIT 20
"""

# Récupération des résultats de la requête SPARQL
results = get_sparql_results(endpoint_url, query)

# Traitement des résultats et création d'un DataFrame
array = [(result["grandevilleLabel"]["value"],
          result["paysLabel"]["value"],
          result["image"]["value"])
        for result in results["results"]["bindings"]]

dataframe = pd.DataFrame(array, columns=["ville", "pays", "image"])
dataframe = dataframe.astype(dtype={"ville": "<U200", "pays": "<U200", "image": "<U200"})

# Affichage du DataFrame pour vérification
dataframe.head()

4. Fonction de téléchargement des images

In [166]:
def download_image(url):
    """
    Télécharge une image depuis une URL et la sauvegarde dans le dossier spécifié.
    
    :param url: URL de l'image à télécharger
    """
    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    
    # Vérification du code de statut de la requête
    # Si la requête a réussi (code 200), on sauvegarde l'image
    if request.status_code == 200:
        filename = os.path.join(IMAGES_DIR, unquote(urlparse(url).path.split('/')[-1]))
        with open(filename, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
        
        print(f"Image sauvegardée : {filename}")
    else:
        print(f"Échec de la récupération de l'image : {request.status_code}")

5. Téléchargement des images

In [ ]:
# Applique la fonction de téléchargement à chaque URL d'image
for image_url in dataframe['image']:
    download_image(image_url)

6. Extraction des métadata EXIF

In [169]:
from PIL import Image
import json

def get_exif_data(image_path):
    """
    Extrait les métadonnées EXIF d'une image.
    
    :param image_path: Chemin vers le fichier image.
    :return: Dictionnaire des métadonnées EXIF.
    """
    try:
        with Image.open(image_path) as img:
            exif_data = img._getexif()
            # Les données EXIF peuvent être None si aucune n'est trouvée
            if exif_data is not None:
                # Convertit les valeurs EXIF en un format plus lisible
                exif = {
                    Image.ExifTags.TAGS[k]: v
                    for k, v in exif_data.items()
                    if k in Image.ExifTags.TAGS and isinstance(v, (str, int, float))
                }
                # Ajoute des informations supplémentaires
                exif['File Size'] = os.path.getsize(image_path)
                exif['Image Format'] = img.format
                exif['Image Size'] = img.size
                exif['Orientation'] = exif.get('Orientation', 'Undefined')
                return exif
            else:
                return {}
    except IOError:
        print(f"Impossible d'ouvrir l'image : {image_path}")
        return {}

7. Sauvegarde des métadata

In [ ]:
# Sauvegarde des métadata dans une liste
metadata = []
for image_filename in os.listdir(IMAGES_DIR):
    image_path = os.path.join(IMAGES_DIR, image_filename)
    exif_data = get_exif_data(image_path)
    metadata.append({
        'Filename': image_filename,
        'Metadata': exif_data
    })

# Sauvegarde des métadonnées dans un fichier JSON
with open('image_metadata.json', 'w') as json_file:
    json.dump(metadata, json_file, indent=4)

print("Métadonnées enregistrées dans image_metadata.json")

### Étiquetage et annotation
Pour cette tâche, vous devez rechercher les sources disposant d'informations supplémentaires comme les balises, les catégories, etc.

Dans cette tâche, vous devrez peut-être étiqueter, annoter et enregistrer des informations sur chaque image. Vous pouvez analyser les images en utilisant des algorithmes de regroupement pour trouver les couleurs prédominantes.

Vous disposez déjà de certaines métadonnées provenant de l'EXIF des images de la précédente tâche. Dans cette tâche, votre objectif est d'obtenir des informations supplémentaires, comme les couleurs prédominantes, les tags. Et si vous demandiez aux utilisateurs de tagger les images ? Par exemple, les noms de couleurs, #cat, #fleur, #sous-fleur, rose etc.

Comment prévoyez-vous de traiter ces tags ? Est-il possible d'automatiser ce processus ?

1. Analyse des couleurs prédominantes

Nous utilisons le KMeans clustering pour trouver les couleurs prédominantes des images. Il est également possible d'utiliser l'analyse en composantes principales (`PCA`) pour réduire le nombre de couleurs dans l'image.

In [176]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt

def find_dominant_colors(image_path, n_colors=5):
    """
    Trouve les couleurs prédominantes dans une image.
    
    :param image_path: Chemin vers le fichier image.
    :param n_colors: Nombre de couleurs prédominantes à trouver.
    :return: Liste des couleurs prédominantes.
    """
    with Image.open(image_path) as img:
        # S'assurer que l'image est en mode RGB
        if img.mode != 'RGB':
            img = img.convert('RGB')
        np_pixels = np.array(img.resize((100, 100))).reshape(-1, 3)
    
    # Réduction de dimensionnalité pour accélérer le processus
    # pca = PCA(n_components=3)
    # pca.fit(np_pixels)
    # pca_pixels = pca.transform(np_pixels)
    
    kmeans = KMeans(n_clusters=n_colors, n_init=10)
    kmeans.fit(np_pixels)
    # kmeans.fit(pca_pixels)
    
    # Récupération des centres de clusters comme couleurs dominates et assurer que les valeurs sont dans l'intervalle [0, 255]
    dominant_colors = np.clip(kmeans.cluster_centers_, 0, 255).astype(int)
    
    # Conversion en hexadécimal
    hex_colors = ['#{:02x}{:02x}{:02x}'.format(*color) for color in dominant_colors]
    
    return hex_colors

2. Récupération automatique des tags

Nous utiliserons l'API Wikimedia Commons pour récupérer les tags associés à une image.

In [177]:
import requests
from bs4 import BeautifulSoup

def get_tags(filename):
    """
    Récupère les tags associés à un fichier Wikimedia Commons.
    
    :param filename: Nom du fichier sur Wikimedia Commons.
    :return: Liste des tags associés.
    """
    start_url = "https://commons.wikimedia.org/wiki/File:"
    url = f"{start_url}{filename}"
    response = requests.get(url)
    tags = []
    
    # Retourne les tags si la requête est réussie
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        category_links = soup.find_all('div', {'id': 'mw-normal-catlinks'})[0].find_all('a', href=True)
        tags = [link.text for link in category_links if link['href'].startswith('/wiki/Category:')]
        
    return tags

3. Récupération de la taille et de l'orientation des images

In [178]:
# Fonction pour obtenir la taille et l'orientation de l'image
def get_image_size_and_orientation(image_path):
    """
    Récupération de la taille et l'orientation
    
    :param image_path: Chemin vers le fichier image.
    :return: Tuple de la taille et de l'orientation.
    """
    with Image.open(image_path) as img:
        width, height = img.size
        orientation = 'square'
        if width > height:
            orientation = 'landscape'
        elif height > width:
            orientation = 'portrait'
    return f"{width}x{height}", orientation

4. Création d'un dataframe pour l'annotation

Nous utiliserons Pandas pour créer un DataFrame qui stockera les informations d'annotation, y compris les tags fournis par les utilisateurs et les couleurs prédominantes.

In [ ]:
import pandas as pd

# Création d'un DataFrame pour les annotations
df_annotations = pd.DataFrame(columns=['Filename', 'Dominant Colors', 'Auto Tags', 'User Tags', 'Image Size', 'Orientation'])

# Ajout des informations dans le DataFrame
for item in metadata:
    filename = item['Filename']
    image_path = os.path.join(IMAGES_DIR, filename)
    
    # Trouve les couleurs prédominantes
    dominant_colors = find_dominant_colors(image_path)
    
    # Trouve les tags associés
    auto_tags = get_tags(filename)
    
    # Trouve les informations de taille et d'orientation
    image_size, orientation = get_image_size_and_orientation(image_path)
    
    # Préparation de la nouvelle ligne à ajouter
    new_row = pd.DataFrame([{
        'Filename': filename,
        'Dominant Colors': dominant_colors,
        'Auto Tags': auto_tags,
        'User Tags': [], # Sera rempli par interaction utilisateur
        'Image Size': image_size,
        'Orientation': orientation
    }])
    
    # Utilisation de pd.concat pour ajouter la nouvelle ligne
    df_annotations = pd.concat([df_annotations, new_row], ignore_index=True)

5. Fonction pour ajouter des tags manuellement

Nous allons créer une interface simple (à l'aide d'un widget Jupyter ou en ligne de commande) pour permettre aux utilisateurs d'ajouter des tags aux images.

In [180]:
def add_tags_to_image(df, index):
    """
    Permet aux utilisateurs d'ajouter des tags à une image.
    
    :param df: DataFrame d'annotations.
    :param index: Index de l'image dans le DataFrame.
    """
    print(f"Image: {df.at[index, 'Filename']}")
    plt.imshow(Image.open(os.path.join(IMAGES_DIR, df.at[index, 'Filename'])))
    plt.axis('off')  # Désactivation des axes
    plt.show()
    
    tags = input("Entrez des tags pour cette image, séparés par des virgules : ").split(',')
    df.at[index, 'User Tags'] = tags

6. Boucle pour l'étiquetage manuel

Nous allons exécuter une boucle qui demande à l'utilisateur d'entrer des tags pour chaque image.

In [ ]:
# Ceci doit être exécuté interactivement dans un environnement Jupyter
for index, row in df_annotations.iterrows():
    add_tags_to_image(df_annotations, index)

5. Sauvegarde des annotations

Finalement, nous allons sauvegarder les annotations dans un fichier JSON.

In [ ]:
# Sauvegarde des annotations
annotations_json = df_annotations.to_json(orient='records', lines=True)
with open('annotations.json', 'w') as file:
    file.write(annotations_json)

print("Annotations enregistrées dans annotations.json")

**Comment prévoyez-vous de traiter ces tags ? Est-il possible d'automatiser ce processus ?**

Pour traiter les tags récupérés automatiquement et ceux entrés par l'utilisateur, nous envisageons d'adopter une approche structurée qui permet à la fois l'automatisation et la flexibilité pour des ajustements manuels.

**1. Automatisation de la récupération des tags**

Comme démontré précédemment, le processus de récupération des tags à partir de Wikimedia Commons est déjà automatisé grâce à la fonction `get_tags`. Cette fonction extrait les catégories (tags) directement associées aux images, servant de tags initiaux. L'automatisation de ce processus garantit que chaque image est associée à des métadonnées pertinentes dès le début, sans intervention manuelle.


**2. Traitement et normalisation des tags**

Après la récupération des tags, un traitement supplémentaire peut être nécessaire pour assurer la cohérence et la pertinence des tags :
- **Normalisation** : Convertir tous les tags en minuscules pour éviter les doublons dus à des différences de casse.
- **Filtrage** : Supprimer les tags non pertinents ou trop généraux basés sur une liste de mots-clés à exclure. Cette liste peut être ajustée selon les besoins spécifiques du projet.
- **Regroupement** : Pour des tags similaires ou synonymes, envisager de les regrouper sous un tag unifié pour simplifier l'analyse et la recherche.
Ces étapes peuvent être partiellement automatisées avec des règles prédéfinies ou des listes de mots-clés, mais elles peuvent également bénéficier d'une revue manuelle pour des cas spécifiques.

Ces étapes peuvent être partiellement automatisées avec des règles prédéfinies ou des listes de mots-clés, mais elles peuvent également bénéficier d'une revue manuelle pour des cas spécifiques.

**3. Intégration des tags utilisateur**

Les tags entrés manuellement par les utilisateurs enrichissent les métadonnées des images avec des perspectives uniques et peuvent combler les lacunes des tags automatiques. Pour intégrer ces tags :
- **Collecte** : Utiliser une interface simple (formulaire web ou widget notebook par exemple), comme montré précédemment, pour permettre aux utilisateurs d'ajouter des tags.
- **Validation** : Bien que les tags des utilisateurs soient précieux, il est important de les valider pour éviter le spam ou les entrées non pertinentes. Cette validation peut être semi-automatisée avec des filtres de mots-clés ou des vérifications de pertinence.
- **Fusion** : Les tags utilisateurs sont fusionnés avec les tags automatiques, en veillant à éliminer les doublons et à maintenir la cohérence de l'ensemble des tags.

**4. Sauvegarde et utilisation des tags**

Une fois les tags traités et validés, ils seront sauvegardés avec les autres métadonnées des images. Ces tags peuvent ensuite être utilisés pour :
- **Recherche et filtrage** : Améliorer les fonctionnalités de recherche et de filtrage dans la base de données d'images.
- **Analyse de données** : Servir de base pour des analyses plus approfondies, comme identifier les tendances dans les images ou comprendre les préférences des utilisateurs.
- **Amélioration du système de recommandation** : Enrichir les données disponibles pour le système de recommandation, permettant des suggestions plus précises et personnalisées.

En résumé, le traitement des tags peut être majoritairement automatisé, surtout pour la récupération et le traitement initial. Cependant, l'intégration des contributions des utilisateurs et certains aspects du filtrage et de la normalisation des tags peuvent bénéficier d'une approche "semi-automatique" qui combine les règles prédéfinies avec une validation manuelle pour assurer la qualité et la pertinence des métadonnées.

### Analyses de données
Demandez à l'utilisateur de sélectionner quelques images et d'ajouter des balises. Pour chaque utilisateur, vous êtes maintenant prêt à construire un profil de préférences d'utilisateur, basé sur cette sélection. Vous pouvez recueillir les informations suivantes manuellement, mais l'objectif de cette tâche consiste à les obtenir en utilisant les images sélectionnées de manière automatisée :
1. Couleurs préférées
2. Orientation de l'image préférée
3. Tailles d'images préférées (vignettes, grandes images, images de taille moyenne images, etc.)
4. Balises favorites
5. ...

Maintenant, grâce à votre connaissance des différents types de classificateurs et les algorithmes de regroupement, quelles informations supplémentaires ajouterez-vous pour chaque image ?

Votre prochain objectif est d'analyser les informations des utilisateurs et leur les images préférées.
Comment avez-vous créé des utilisateurs aléatoires ? Combien d'utilisateurs avez-vous créer ? Quelles informations avez-vous stockées pour chaque utilisateur ? Quels types d'analyses avez-vous effectuées ?

1. Sélection des images par l'utilisateur

Pour commencer, nous permettons à l'utilisateur de sélectionner un ensemble d'images. Ces images serviront de base pour construire le profil de préférences de l'utilisateur.

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import display
from PIL import Image

def user_select_images(df):
    selected_indices = []
    for index, row in df.iterrows():
        # Affichage de l'image
        image_path = os.path.join(IMAGES_DIR, row['Filename'])
        img = Image.open(image_path)
        plt.imshow(img)
        plt.axis('off')  # Désactivation des axes
        plt.show()
        
        # Demande à l'utilisateur s'il souhaite sélectionner l'image
        user_input = input(f"Voulez-vous sélectionner l'image {row['Filename']} ? (y/n) : ").lower()
        if user_input == 'y':
            selected_indices.append(index)
            
        # Option pour arrêter le processus si l'utilisateur ne veut plus sélectionner d'images
        continue_choice = input("Voulez-vous continuer à voir plus d'images ? (y/n) : ").lower()
        if continue_choice != 'y':
            break
            
    return selected_indices

# Utilisation de la fonction
selected_images_indices = user_select_images(df_annotations)
print("Indices des images sélectionnées :", selected_images_indices)

Ce script parcourt toutes les lignes du DataFrame contenant les informations sur les images, affiche chaque image à l'aide de matplotlib, et demande à l'utilisateur s'il souhaite sélectionner l'image affichée. Si l'utilisateur répond par 'y' (yes), l'indice de l'image est ajouté à la liste des indices sélectionnés. L'utilisateur a également la possibilité d'arrêter le processus à tout moment s'il ne souhaite plus sélectionner d'autres images.

2. Analyse des préférences de l'utilisateur

Nous allons analyser les préférences de l'utilisateur basées sur les images sélectionnées. Cela inclut les couleurs préférées, l'orientation de l'image, les tailles d'images et les balises favorites.

    1. Couleurs préférées 

In [184]:
def get_preferred_colors(df, indices):
    """
    Récupère les couleurs préférées des images sélectionnées.
    
    :param df: DataFrame d'annotations.
    :param indices: Indices des images sélectionnées.
    :return: Liste des couleurs préférées.
    """
    color_preferences = []
    for index in indices:
        color_preferences.extend(df.loc[index, 'Dominant Colors'])
    return list(set(color_preferences))

preferred_colors = get_preferred_colors(df_annotations, selected_images_indices)

    2. Orientation de l'image préférée

In [ ]:
def get_preferred_orientation(df, selected_indices):
    """
    Récupère l'orientation préférée des images sélectionnées.
    
    :param df: DataFrame d'annotations.
    :param selected_indices: Indices des images sélectionnées.
    :return: Orientation préférée.
    """
    orientation_counts = df.loc[selected_indices, 'Orientation'].value_counts()
    preferred_orientation = orientation_counts.idxmax()
    return preferred_orientation

# Utilisation de la fonction avec les indices d'images sélectionnés par l'utilisateur
preferred_orientation = get_preferred_orientation(df_annotations, selected_images_indices)
print("Orientation préférée :", preferred_orientation)

Cette fonction compte le nombre d'occurrences de chaque orientation parmi les images sélectionnées et détermine l'orientation la plus fréquente comme étant la préférée de l'utilisateur.

    3. Tailles d'images préférées

In [ ]:
def categorize_image_size(width, height):
    """
    Catégorise la taille de l'image en petite, moyenne ou grande.
    
    :param width: Largeur de l'image.
    :param height: Hauteur de l'image.
    :return: Catégorie de taille.
    """
    pixels = width * height
    if pixels < 640 * 480:
        return 'small'
    elif pixels <= 1280 * 1024:
        return 'medium'
    else:
        return 'large'

def get_preferred_size(df, selected_indices):
    """
    Récupère la taille d'image préférée des images sélectionnées.
    
    :param df: DataFrame d'annotations.
    :param selected_indices: Indices des images sélectionnées.
    :return: Taille d'image préférée.
    """
    sizes = [categorize_image_size(*map(int, df.loc[index, 'Image Size'].split('x'))) for index in selected_indices]
    size_counts = pd.Series(sizes).value_counts()
    preferred_size = size_counts.idxmax()
    return preferred_size

# Utilisation de la fonction avec les indices d'images sélectionnés par l'utilisateur
preferred_size = get_preferred_size(df_annotations, selected_images_indices)
print("Taille d'image préférée :", preferred_size)

Cette approche décompose la chaîne Image Size en largeur et hauteur, les convertit en entiers, calcule le nombre total de pixels, et catégorise ensuite la taille de l'image. La taille la plus fréquente parmi les sélections de l'utilisateur est considérée comme sa préférée.

    4. Balises favorites

In [187]:
def get_favorite_tags(df, indices):
    """
    Récupère les tags préférés des images sélectionnées.
    
    :param df: DataFrame d'annotations.
    :param indices: Indices des images sélectionnées.
    :return: Liste des tags préférés.
    """
    tags = []
    for index in indices:
        tags.extend(df.loc[index, 'Auto Tags'] + df.loc[index, 'User Tags'])
    return list(set(tags))

favorite_tags = get_favorite_tags(df_annotations, selected_images_indices)

3. Construction du profil de préférences

Après avoir recueilli les informations sur les préférences de l'utilisateur, nous assemblons ces informations dans un profil de préférences.

In [ ]:
user_preferences_profile = {
    'Preferred Colors': preferred_colors,
    'Preferred Orientation': preferred_orientation,
    'Preferred Size': preferred_size,
    'Favorite Tags': favorite_tags
}

print(user_preferences_profile)

**Maintenant, grâce à votre connaissance des différents types de classificateurs et les algorithmes de regroupement, quelles informations supplémentaires ajouterez-vous pour chaque image ?**

Avec un profil de préférences utilisateur déjà établi, basé sur les couleurs dominantes, l'orientation, la taille des images, et les tags, nous pouvons envisager d'ajouter plusieurs dimensions supplémentaires pour enrichir notre analyse et améliorer le système de recommandation. Voici quelques idées d'informations supplémentaires qui pourraient être pertinentes :

**1. Complexité visuelle**

La complexité visuelle d'une image peut être un indicateur intéressant de préférence. Certaines personnes peuvent préférer des images simples et épurées, tandis que d'autres peuvent être attirées par des images plus complexes et détaillées. Des mesures de texture ou de variété de couleurs peuvent être utilisées pour évaluer cette complexité.

**2. Sentiments et Emotions**

Les images évoquent des sentiments et des émotions. L'analyse sentimentale des images, bien que plus complexe, peut permettre de classer les images en catégories émotionnelles (joyeuse, triste, paisible, etc.). Cela pourrait se faire par des classificateurs entraînés sur des ensembles de données annotés avec des étiquettes émotionnelles.

**3. Sujets et Objets**

Identifier les sujets ou objets principaux présents dans les images (animaux, bâtiments, paysages naturels, etc.) peut aider à affiner les préférences. Ceci peut être réalisé grâce à la vision par ordinateur et l'apprentissage profond, par exemple avec des modèles pré-entraînés comme ceux disponibles dans TensorFlow ou PyTorch.

**4. Popularité et Engagement**

Pour les images provenant de sources en ligne, des métriques de popularité et d'engagement (nombre de vues, likes, partages) pourraient également être pertinentes. Ces informations peuvent aider à identifier les images qui résonnent le plus avec le public en général.

**5. Contexte Temporel ou Saisonnier**

Le moment de l'année ou le contexte temporel de l'image peut influencer les préférences. Par exemple, des images de paysages enneigés pourraient être plus attrayantes en hiver. Cette information peut être déduite des tags ou des métadonnées de l'image.

Pour analyser les informations des utilisateurs et leurs images préférées, nous allons simuler un ensemble d'utilisateurs avec des préférences générées de manière aléatoire. Cela nous permettra de montrer comment les données utilisateur peuvent être analysées et utilisées pour des recommandations.

1. Création d'utilisateurs aléatoires

D'abord, créons un ensemble d'utilisateurs aléatoires. Pour chaque utilisateur, nous allons générer des préférences aléatoires pour les couleurs, l'orientation des images, la taille des images, et les tags.

Les informations suivantes sont stockées pour chaque utilisateur :
- **user_id** : Un identifiant unique pour l'utilisateur.
- **Preferred Colors** : Un ensemble de couleurs préférées sélectionnées aléatoirement.
- **Preferred Orientation** : L'orientation préférée de l'image (portrait, paysage, ou carré).
- **Preferred Size** : La taille préférée de l'image (petite, moyenne, ou grande).
- **Favorite Tags** : Un ensemble de tags qui représentent les intérêts de l'utilisateur.

In [ ]:
import random

# Nombre d'utilisateurs à générer
num_users = 5 # A modifier selon le nombre d'utilisateurs souhaité

# Création des utilisateurs avec des préférences aléatoires
users_profiles = []

for i in range(1, num_users + 1):
    # Sélection aléatoire d'images
    selected_indices = random.sample(range(len(df_annotations)), k=random.randint(1, len(df_annotations))) # Entre 1 et le nombre total d'images
    
    user_profile = {
        'user_id': i,
        'Preferred Colors': get_preferred_colors(df_annotations, selected_indices),
        'Preferred Orientation': get_preferred_orientation(df_annotations, selected_indices),
        'Preferred Size': get_preferred_size(df_annotations, selected_indices),
        'Favorite Tags': get_favorite_tags(df_annotations, selected_indices)
    }
    users_profiles.append(user_profile)

# Affichage des préférences des utilisateurs générés
users_preferences_df = pd.DataFrame(users_profiles)
users_preferences_df

2. Encodage des préférences

Les préférences des utilisateurs, en particulier les tags et les couleurs préférés, sont des données catégorielles qui peuvent être mieux traitées par les algorithmes d'apprentissage automatique après encodage. Pour les tags et les couleurs, nous pouvons utiliser l'encodage One-Hot.

In [204]:
from sklearn.preprocessing import MultiLabelBinarizer

# Vérification que les colonnes existent
if 'Favorite Tags' in users_preferences_df.columns and 'Preferred Colors' in users_preferences_df.columns:
    # Encodage One-Hot pour les tags favoris
    mlb_tags = MultiLabelBinarizer()
    tags_encoded = mlb_tags.fit_transform(users_preferences_df['Favorite Tags'])
    tags_encoded_df = pd.DataFrame(tags_encoded, columns=mlb_tags.classes_)
    
    # Encodage One-Hot pour les couleurs préférées
    mlb_colors = MultiLabelBinarizer()
    colors_encoded = mlb_colors.fit_transform(users_preferences_df['Preferred Colors'])
    colors_encoded_df = pd.DataFrame(colors_encoded, columns=mlb_colors.classes_)
    
    # Concaténation avec le DataFrame original sans supprimer les colonnes originales pour éviter KeyError
    users_preferences_df = pd.concat([users_preferences_df, tags_encoded_df, colors_encoded_df], axis=1)
else:
    print("Erreur : Colonnes 'Favorite Tags' ou 'Preferred Colors' manquantes.")

# Affichage des préférences avec les colonnes encodées
# users_preferences_df

3. Clustering des utilisateurs

Nous pouvons utiliser un algorithme de clustering pour regrouper les utilisateurs en fonction de leurs préférences. Cela pourrait révéler des patterns dans les préférences des utilisateurs et permettre des recommandations ciblées.

    1. Vérification et Sélection des Colonnes

In [ ]:
# Liste des colonnes à exclure explicitement, incluant l'ID utilisateur et toute autre colonne non numérique
columns_to_exclude = ['user_id', 'Preferred Orientation', 'Preferred Size', 'Favorite Tags', 'Preferred Colors']

# Sélection des colonnes pour le clustering, en excluant les colonnes non numériques ou contenant des listes
columns_for_clustering = [col for col in users_preferences_df.columns if col not in columns_to_exclude]

print("Colonnes sélectionnées pour le clustering :", columns_for_clustering)

    2. Exécution de KMeans

In [206]:
# Application du KMeans avec les colonnes correctement sélectionnées
kmeans = KMeans(n_clusters=n_clusters, random_state=0, n_init=10).fit(users_preferences_df[columns_for_clustering])

# Ajout des labels de cluster au DataFrame
users_preferences_df['Cluster'] = kmeans.labels_

4. Analyse des clusters

Une fois les clusters d'utilisateurs identifiés, nous pouvons analyser les caractéristiques de chaque cluster pour identifier les insights spécifiques, et comprendre les tendances et les similitudes entre les utilisateurs. Cela peut aider à personnaliser les recommandations en fonction des différents groupes d'utilisateurs.

In [ ]:
for i in range(n_clusters):
    # Sélection du cluster actuel
    cluster = users_preferences_df[users_preferences_df['Cluster'] == i]
    print(f"Analyse du Cluster {i} :")
    print(f"Nombre d'utilisateurs : {cluster.shape[0]}")
    print("Utilisateur ID :", ', '.join(cluster['user_id'].astype(str).values))

    # Distribution des préférences d'orientation
    orientation_distribution = cluster['Preferred Orientation'].value_counts(normalize=True) * 100
    print("\nDistribution des préférences d'orientation (%) : ", orientation_distribution.to_string())

    # Distribution des préférences de taille
    size_distribution = cluster['Preferred Size'].value_counts(normalize=True) * 100
    print("\nDistribution des préférences de taille (%) : ", size_distribution.to_string())

    # Tags communs et leur fréquence
    all_tags_in_cluster = sum(cluster['Favorite Tags'].tolist(), [])
    tags_freq = pd.Series(all_tags_in_cluster).value_counts().head(5)
    print("\nTags communs :")
    print(tags_freq.to_string())

    # Supposons que les préférences de couleurs sont enregistrées de manière similaire aux tags
    if 'Preferred Colors' in cluster.columns:
        all_colors_in_cluster = sum(cluster['Preferred Colors'].tolist(), [])
        if all_colors_in_cluster:  # Vérification que la liste n'est pas vide
            colors_freq = pd.Series(all_colors_in_cluster).value_counts().head(5)
            print("\nCouleurs communes :")
            print(colors_freq.to_string())
        else:
            print("\nCouleurs communes : Informations sur les couleurs non disponibles")
    else:
        print("\nCouleurs communes : Informations sur les couleurs non disponibles")

    print("\n----------------------------------------\n")

#### **Explications**

- **Nombre d'utilisateurs et ID** : Nous affichons le nombre d'utilisateurs dans chaque cluster et leurs identifiants pour faciliter le suivi et l'identification des segments d'utilisateurs.
- **Préférences d'orientation et de taille** : Nous résumons les préférences d'orientation et de taille les plus courantes dans chaque cluster. Plus précisémment, pour chaque cluster, nous calculons la distribution des préférences d'orientation et de taille parmi les utilisateurs. L'utilisation de `value_counts(normalize=True)` permet de calculer les proportions, que nous multiplions par 100 pour obtenir des pourcentages.
- **Tags communs** : Nous identifions les tags les plus fréquents dans le cluster et affichons leur fréquence. Cela donne une idée des intérêts ou des caractéristiques les plus communs parmi les utilisateurs du cluster.
- **Couleurs communes** : Si les préférences de couleurs sont disponibles, nous calculons également les couleurs les plus fréquentes dans chaque cluster. Ceci est fait de manière similaire aux tags.

#### **Création des utilisateurs aléatoires**

- **Comment avez-vous créé des utilisateurs aléatoires ?**

Nous avons généré des utilisateurs aléatoires en simulant des sélections d'images pour chaque utilisateur. Pour chaque utilisateur simulé, un ensemble aléatoire d'images a été sélectionné, et à partir de ces images, des préférences telles que les couleurs préférées, l'orientation de l'image préférée, les tailles d'images préférées, et les tags favoris ont été dérivées.

- **Combien d'utilisateurs avez-vous créé ?**

Nous avons simulé un total de `5` utilisateurs, bien que ce nombre puisse être ajusté en fonction des besoins spécifiques de l'analyse ou des tests.

#### **Informations stockées pour chaque utilisateur**

- **Quelles informations avez-vous stockées pour chaque utilisateur ?**

Pour chaque utilisateur, les informations suivantes ont été stockées :
- Les `couleurs préférées`, déterminées à partir des images sélectionnées.
- L'`orientation` de l'image préférée (paysage, portrait, carré).
- La `taille` d'image préférée (petite, moyenne, grande), basée sur les dimensions des images sélectionnées.
- Les `tags favoris`, extraits à la fois automatiquement des images sélectionnées et à travers des entrées manuelles simulées.
- Un `identifiant de cluster` attribué après avoir effectué un `clustering KMeans` sur les utilisateurs basé sur leurs préférences encodées.


#### **Types d'analyses effectuées**

- **Quels types d'analyses avez-vous effectuées ?**

Plusieurs types d'analyses ont été effectuées sur les données simulées des utilisateurs :
- **Clustering des utilisateurs** : Utilisation de l'algorithme `KMeans` pour grouper les utilisateurs en clusters basés sur leurs préférences encodées en `One-Hot`.
- **Analyse des préférences par cluster** : Pour chaque cluster, nous avons analysé les préférences communes, telles que l'orientation et la taille d'image préférées, ainsi que les tags et couleurs les plus fréquents. Cette analyse a permis de dégager des tendances et des caractéristiques communes au sein de chaque groupe d'utilisateurs.
- **Distribution des préférences** : Pour enrichir l'analyse, nous avons calculé et affiché la distribution des préférences d'orientation, de taille, des tags et couleurs préférés au sein de chaque cluster, offrant une vue plus nuancée des tendances générales.

Ces analyses fournissent une compréhension approfondie des segments d'utilisateurs, en identifiant des patterns et des préférences communes qui peuvent être utilisées pour améliorer le système de recommandation d'images, en personnalisant les suggestions en fonction des profils de préférences détectés.

### Visualisation des données
Dans cette tâche, votre objectif est de visualiser les différentes caractéristiques de toutes les images
téléchargées.
1. Le nombre d'images disponibles pour chaque année
2. Le nombre d'images disponibles pour les différents types : taille de l'image, l'orientation des images, les modèles d'appareils photo, etc.
3. Caractéristiques des couleurs

Les utilisateurs peuvent également visualiser les informations ci-dessus relatives à leurs images préférées.
Dans cette tâche, vous devez également ajouter une fonctionnalité permettant aux utilisateurs de
visualiser les informations liées à leur propre profil d'utilisateur.